In [1]:
# =========================
# X - IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_plots import *

# =========================


c:\Users\sal_f\anaconda3\envs\env_ggs0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = GGS_config_static(date=dt.datetime.now(timezone.utc))
sub_directory_data = GGS_config_output(config, path="default")
sub_directory_plots = GGS_config_output(config, path="default")

datetime_index = "2024-02-18 00:00:00.000000+00:00"




### Glider Guidance System (GGS) Configuration ###

Glider name: Yucatan

Execution date: 2024-03-01 17:09:22.345385+00:00

Max Depth: 10

Extent:
  Boundary 1: (15, -93)
  Boundary 2: (30, -77)

GPS_coords:
  GPS_coord 1: (39.4, -74.2)
  GPS_coord 2: (39.3, -71.2)



### Glider Guidance System (GGS) Configuration ###

Glider name: Yucatan

Execution date: 2024-03-01 17:09:22.345385+00:00

Max Depth: 10

Extent:
  Boundary 1: (15, -93)
  Boundary 2: (30, -77)

GPS_coords:
  GPS_coord 1: (39.4, -74.2)
  GPS_coord 2: (39.3, -71.2)



In [3]:
# format_title_datetime(datetime_index)



'2024-02-18 00:00'

In [ ]:
rtofs = RTOFS()
rtofs.rtofs_load(config, datetime_index, subset_extent=True, subset_depth=True) # [subset] OPTIONS: True = Subset Data, False = Do Not Subset Data
rtofs.rtofs_save(config, sub_directory_data, save_data=False, save_qc=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File
rtofs_data = rtofs.data
rtofs_qc = rtofs.qc

depth_average_rtofs, bin_average_rtofs = interpolate_rtofs(config, sub_directory_data, rtofs_data, chunk=False, save_depth_average=False, save_bin_average=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File


In [ ]:
cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
cmems.cmems_load(config, datetime_index, subset_extent=True, subset_depth=True) # [subset] OPTIONS: True = Subset Data, False = Do Not Subset Data
cmems.cmems_save(config, sub_directory_data, save_data=False, save_qc=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File
cmems_data = cmems.data
cmems_qc = cmems.qc

depth_average_cmems, bin_average_cmems = interpolate_cmems(config, sub_directory_data, cmems_data, chunk=False, save_depth_average=False, save_bin_average=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File


In [ ]:
gofs = GOFS()
gofs.gofs_load(config, datetime_index, subset_extent=True, subset_depth=True) # [subset] OPTIONS: True = Subset Data, False = Do Not Subset Data
gofs.gofs_save(config, sub_directory_data, save_data=False, save_qc=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File
gofs_data = gofs.data
gofs_qc = gofs.qc

depth_average_gofs, bin_average_gofs = interpolate_gofs(config, sub_directory_data, gofs_data, chunk=False, save_depth_average=True, save_bin_average=False) # [save] OPTIONS: True = Save Data File, False = Do Not Save Data File

# latitude_qc='20.30'
# longitude_qc='-86.50'
# GGS_plot_magnitude(config, sub_directory_plots, depth_average_gofs, latitude_qc, longitude_qc, density=2, show_gliders=True, show_route=False, show_qc=False, manual_extent=None)
# GGS_plot_threshold(config, sub_directory_plots, depth_average_gofs, latitude_qc, longitude_qc, density=2, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, show_gliders=True, show_route=False, show_qc=False, manual_extent=None)


In [ ]:
### FUNCTION:
def format_subplot_titles(fig, config, model_data, title):
    
    '''
    Sets the main title, subtitle, and suptitle for a plot with correct positioning.

    Args:
    - ax (matplotlib.axes.Axes): The axes object to set the main title on.
    - fig (matplotlib.figure.Figure): The figure object to set subtitles and suptitles on.
    - config (dict): Configuration dictionary containing plot settings and metadata.
    - model_data (xarray.Dataset): Dataset used in the plot, to derive the datetime title.
    - title (str): Custom title text for the main title of the plot.
    - model (str): Model source identifier, used to prefix the subtitle.

    Returns:
    - None
    '''

    fig.suptitle(title, fontsize=26, fontweight='bold', ha='center', y=0.95)
    
    title_datetime = format_model_datetime(model_data)
    subtitle_text = f"{title_datetime} UTC"
    fig.text(0.5, 0.92, subtitle_text, fontsize=22, ha='center', va='top')
    
    suptitle_text = f"Generated by the Glider Guidance System (GGS) - {config['glider_name']}"
    fig.text(0.5, 0.05, suptitle_text, fontsize=20, ha='center', va='top', color='gray')


In [ ]:
def format_subplot_headers(axes, fig, subplot_titles):

    '''
    Adds subplot headers as bolded titles above each subplot row figure.

    Args:
    - axes (numpy.ndarray): Array of axes objects to add headers to.
    - fig (matplotlib.figure.Figure): Figure object to add text to.
    - subplot_titles (list): List of strings to use as header titles.

    Returns:
    - None
    '''

    if axes.ndim == 1:
        axes = np.array([axes])

    for ax, model_name in zip(axes[:, 0], subplot_titles):
        bbox = ax.get_position()
        text_x = bbox.x0
        text_y = bbox.y1 + 0.02
        
        fig.text(text_x, text_y, model_name, fontsize=14, fontweight='bold', ha='left')


In [ ]:
def calculate_gridpoint(model_data, target_lat, target_lon):
    
    '''
    Calculate the nearest XY gridpoint in a model dataset to the input latitude and longitude, accommodating both 1D and 2D lat/lon arrays.

    Args:
    - model_data (xarray.Dataset): The model dataset.
    - target_lat (float): The target latitude.
    - target_lon (float): The target longitude.

    Returns:
    - (y_index, x_index) (tuple): The indices of the nearest point in the dataset.
    - (lat_index, lon_index) (tuple): The coordinates of the nearest point in the dataset.
    '''
    
    if 'lat' in model_data.dims and 'lon' in model_data.dims:
        lat = model_data['lat'].values
        lon = model_data['lon'].values
        lon_grid, lat_grid = np.meshgrid(lon, lat)
        squared_dist = (lat_grid - target_lat)**2 + (lon_grid - target_lon)**2
    else:
        lat_diff = model_data['lat'] - target_lat
        lon_diff = model_data['lon'] - target_lon
        squared_dist = lat_diff**2 + lon_diff**2
    
    y_index, x_index = np.unravel_index(squared_dist.argmin(), squared_dist.shape)
    
    if 'lat' in model_data.dims and 'lon' in model_data.dims:
        lat_index = lat[y_index]
        lon_index = lon[x_index]
    else:
        lat_index = model_data['lat'].isel(y=y_index, x=x_index).values
        lon_index = model_data['lon'].isel(y=y_index, x=x_index).values

    return (y_index, x_index), (lat_index, lon_index)


In [ ]:
def GGS_plot_profiles(config, directory, latitude_qc, longitude_qc, threshold=0.5, rtofs_datasets=None, cmems_datasets=None, gofs_datasets=None):

    '''
    Produce quality control profiles for 'u', 'v', 'magnitude', and 'direction' data at the specified point of interest.

    Args:
    - config (dict): The configuration dictionary.
    - directory (str): The directory to save the plot.
    - latitude_qc (float): The latitude of the point of interest.
    - longitude_qc (float): The longitude of the point of interest.
    - threshold (float): The threshold value for the shading.
    - rtofs_datasets (tuple): The RTOFS datasets (model data, depth-averaged data, bin-averaged data)
    - cmems_datasets (tuple): The CMEMS datasets (model data, depth-averaged data, bin-averaged data)
    - gofs_datasets (tuple): The GOFS datasets (model data, depth-averaged data, bin-averaged data)

    Returns:
    - None
    '''

    # INITIALIZATION
    print(f"\n### CREATING PROFILE PLOT ###\n")
    start_time = print_starttime()

    latitude_qc = float(latitude_qc)
    longitude_qc = float(longitude_qc)

    # DATASET EXTRACTION
    if rtofs_datasets is not None:
        rtofs_data, depth_average_rtofs, bin_average_rtofs = rtofs_datasets

        (y_rtofs_model, x_rtofs_model), _ = calculate_gridpoint(rtofs_data, latitude_qc, longitude_qc)
        (y_rtofs_avg, x_rtofs_avg), _ = calculate_gridpoint(depth_average_rtofs, latitude_qc, longitude_qc)
        (y_rtofs_bin, x_rtofs_bin), _ = calculate_gridpoint(bin_average_rtofs, latitude_qc, longitude_qc)
        
        rtofs_model_u = rtofs_data['u'].isel(y=y_rtofs_model, x=x_rtofs_model).values
        rtofs_avg_u = depth_average_rtofs['u_depth_avg'].isel(y=y_rtofs_avg, x=x_rtofs_avg).values
        rtofs_avg_u = rtofs_avg_u.item()
        rtofs_bin_u = bin_average_rtofs['u_bin_avg'].isel(y=y_rtofs_bin, x=x_rtofs_bin).values
        rtofs_bin_u1d = rtofs_bin_u[0]

        rtofs_model_v = rtofs_data['v'].isel(y=y_rtofs_model, x=x_rtofs_model).values
        rtofs_avg_v = depth_average_rtofs['v_depth_avg'].isel(y=y_rtofs_avg, x=x_rtofs_avg).values
        rtofs_avg_v = rtofs_avg_v.item()
        rtofs_bin_v = bin_average_rtofs['v_bin_avg'].isel(y=y_rtofs_bin, x=x_rtofs_bin).values
        rtofs_bin_v1d = rtofs_bin_v[0]

        rtofs_avg_mag = depth_average_rtofs['mag_depth_avg'].isel(y=y_rtofs_avg, x=x_rtofs_avg).values
        rtofs_avg_mag = rtofs_avg_mag.item()
        rtofs_bin_mag = bin_average_rtofs['mag_bin_avg'].isel(y=y_rtofs_bin, x=x_rtofs_bin).values
        rtofs_bin_mag1d = rtofs_bin_mag[0]

        rtofs_bin_dir = bin_average_rtofs['dir_bin_avg'].isel(y=y_rtofs_bin, x=x_rtofs_bin).values
        rtofs_bin_dir = np.mod(rtofs_bin_dir + 180, 360) - 180

        rtofs_max_depth = rtofs_data.depth.max().item()
        rtofs_max_bins = rtofs_max_depth + 1
        rtofs_bin_depths = np.arange(rtofs_max_bins)

    if cmems_datasets is not None:
        cmems_data, depth_average_cmems, bin_average_cmems = cmems_datasets
        
        (y_cmems_model, x_cmems_model), _ = calculate_gridpoint(cmems_data, latitude_qc, longitude_qc)
        (y_cmems_avg, x_cmems_avg), _ = calculate_gridpoint(depth_average_cmems, latitude_qc, longitude_qc)
        (y_cmems_bin, x_cmems_bin), _ = calculate_gridpoint(bin_average_cmems, latitude_qc, longitude_qc)

        cmems_model_u = cmems_data['u'].isel(lat=y_cmems_model, lon=x_cmems_model).values
        cmems_avg_u = depth_average_cmems['u_depth_avg'].isel(lat=y_cmems_avg, lon=x_cmems_avg).values
        cmems_avg_u = cmems_avg_u.item()
        cmems_bin_u = bin_average_cmems['u_bin_avg'].isel(lat=y_cmems_bin, lon=x_cmems_bin).values
        cmems_bin_u1d = cmems_bin_u[0]

        cmems_model_v = cmems_data['v'].isel(lat=y_cmems_model, lon=x_cmems_model).values
        cmems_avg_v = depth_average_cmems['v_depth_avg'].isel(lat=y_cmems_avg, lon=x_cmems_avg).values
        cmems_avg_v = cmems_avg_v.item()
        cmems_bin_v = bin_average_cmems['v_bin_avg'].isel(lat=y_cmems_bin, lon=x_cmems_bin).values
        cmems_bin_v1d = cmems_bin_v[0]

        cmems_avg_mag = depth_average_cmems['mag_depth_avg'].isel(lat=y_cmems_avg, lon=x_cmems_avg).values
        cmems_avg_mag = cmems_avg_mag.item()
        cmems_bin_mag = bin_average_cmems['mag_bin_avg'].isel(lat=y_cmems_bin, lon=x_cmems_bin).values
        cmems_bin_mag1d = cmems_bin_mag[0]

        cmems_bin_dir = bin_average_cmems['dir_bin_avg'].isel(lat=y_cmems_bin, lon=x_cmems_bin).values
        cmems_bin_dir = np.mod(cmems_bin_dir + 180, 360) - 180

        cmems_max_depth = cmems_data.depth.max().item()
        cmems_max_bins = cmems_max_depth + 1
        cmems_bin_depths = np.arange(cmems_max_bins)

    if gofs_datasets is not None:
        gofs_data, depth_average_gofs, bin_average_gofs = gofs_datasets
        
        (y_gofs_model, x_gofs_model), _ = calculate_gridpoint(gofs_data, latitude_qc, longitude_qc)
        (y_gofs_avg, x_gofs_avg), _ = calculate_gridpoint(depth_average_gofs, latitude_qc, longitude_qc)
        (y_gofs_bin, x_gofs_bin), _ = calculate_gridpoint(bin_average_gofs, latitude_qc, longitude_qc)

        gofs_model_u = gofs_data['u'].isel(lat=y_gofs_model, lon=x_gofs_model).values
        gofs_avg_u = depth_average_gofs['u_depth_avg'].isel(lat=y_gofs_avg, lon=x_gofs_avg).values
        gofs_avg_u = gofs_avg_u.item()
        gofs_bin_u = bin_average_gofs['u_bin_avg'].isel(lat=y_gofs_bin, lon=x_gofs_bin).values
        gofs_bin_u1d = gofs_bin_u[0]

        gofs_model_v = gofs_data['v'].isel(lat=y_gofs_model, lon=x_gofs_model).values
        gofs_avg_v = depth_average_gofs['v_depth_avg'].isel(lat=y_gofs_avg, lon=x_gofs_avg).values
        gofs_avg_v = gofs_avg_v.item()
        gofs_bin_v = bin_average_gofs['v_bin_avg'].isel(lat=y_gofs_bin, lon=x_gofs_bin).values
        gofs_bin_v1d = gofs_bin_v[0]

        gofs_avg_mag = depth_average_gofs['mag_depth_avg'].isel(lat=y_gofs_avg, lon=x_gofs_avg).values
        gofs_avg_mag = gofs_avg_mag.item()
        gofs_bin_mag = bin_average_gofs['mag_bin_avg'].isel(lat=y_gofs_bin, lon=x_gofs_bin).values
        gofs_bin_mag1d = gofs_bin_mag[0]

        gofs_bin_dir = bin_average_gofs['dir_bin_avg'].isel(lat=y_gofs_bin, lon=x_gofs_bin).values
        gofs_bin_dir = np.mod(gofs_bin_dir + 180, 360) - 180

        gofs_max_depth = gofs_data.depth.max().item()
        gofs_max_bins = gofs_max_depth + 1
        gofs_bin_depths = np.arange(gofs_max_bins)

    # PLOTTING SETUP
    fig, axes = plt.subplots(3, 4, figsize=(20, 25), gridspec_kw={'height_ratios': [1, 1, 1], 'hspace': 0.3})

    if rtofs_datasets is not None:
        # U-VELOCITY PROFILE
        axes[0,0].scatter(rtofs_model_u, rtofs_data.depth, marker='x', color='black', s=100, label='Model Datapoint', alpha=1.0, zorder=3)
        axes[0,0].scatter(rtofs_bin_u, rtofs_bin_depths, label='1m Interpolation', color='cyan', alpha=1.0, zorder=2)
        axes[0,0].axvline(x=rtofs_avg_u, label=f'Depth Average = [{rtofs_avg_u:.2f}]', color='darkcyan', linestyle='--', linewidth=2, zorder=1)
        axes[0,0].set_xlabel('u Velocity (m/s)', fontsize=12, fontweight='bold')
        axes[0,0].set_ylabel('Depth (m)', fontsize=12, fontweight='bold')
        axes[0,0].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[0,0].invert_yaxis()
    
        # V-VELOCITY PROFILE
        axes[0,1].scatter(rtofs_model_v, rtofs_data.depth, label='Model Datapoint', marker='x', color='black', s=100, alpha=1.0, zorder=3)
        axes[0,1].scatter(rtofs_bin_v, rtofs_bin_depths, label='1m Interpolation', color='orange', alpha=1.0, zorder=2)
        axes[0,1].axvline(x=rtofs_avg_v, label=f'Depth Avgerage = [{rtofs_avg_v:.2f}]', color='darkorange', linestyle='--', linewidth=2, zorder=1)
        axes[0,1].set_xlabel('v Velocity (m/s)', fontsize=12, fontweight='bold')
        axes[0,1].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[0,1].invert_yaxis()

        # MAGNITUDE PROFILE
        axes[0,2].scatter(rtofs_bin_mag, rtofs_bin_depths, label='1m Interpolation', color='green', alpha=1.0, zorder=2)
        axes[0,2].set_xlabel('Current Magnitude (m/s)', fontsize=12, fontweight='bold')
        axes[0,2].axvline(x=rtofs_avg_mag, label=f'Depth Avgerage = [{rtofs_avg_mag:.2f}]', color='darkgreen', linestyle='--', linewidth=2, zorder=1)
        axes[0,2].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[0,2].invert_yaxis()

        # DIRECTION PROFILE
        axes[0,3].scatter(rtofs_bin_dir, rtofs_bin_depths, label='1m Interpolation', color='purple', alpha=1.0, zorder=2)
        axes[0,3].set_xlabel('Current Direction (degrees)', fontsize=12, fontweight='bold')
        axes[0,3].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[0,3].invert_yaxis()
    
        # THRESHOLDS
        shading_colors = ['cyan', 'orange', 'lawngreen']
        for i, (data_1d, color) in enumerate(zip([rtofs_bin_u1d, rtofs_bin_v1d, rtofs_bin_mag1d], shading_colors)):
            plot_profile_thresholds(axes[0, i], data_1d, threshold, color)

        # LEGEND
        for row in axes:
            for ax in row:
                handles, labels = ax.get_legend_handles_labels()
                if labels:
                    ax.legend(handles, labels, loc='lower center', facecolor='lightgrey', edgecolor='black', framealpha=1.0)

    if cmems_datasets is not None:
        # U-VELOCITY PROFILE
        axes[1,0].scatter(cmems_model_u, cmems_data.depth, marker='x', color='black', s=100, label='Model Datapoint', alpha=1.0, zorder=3)
        axes[1,0].scatter(cmems_bin_u, cmems_bin_depths, label='1m Interpolation', color='cyan', alpha=1.0, zorder=2)
        axes[1,0].axvline(x=cmems_avg_u, label=f'Depth Average = [{cmems_avg_u:.2f}]', color='darkcyan', linestyle='--', linewidth=2, zorder=1)
        axes[1,0].set_xlabel('u Velocity (m/s)', fontsize=12, fontweight='bold')
        axes[1,0].set_ylabel('Depth (m)', fontsize=12, fontweight='bold')
        axes[1,0].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[1,0].invert_yaxis()

        # V-VELOCITY PROFILE
        axes[1,1].scatter(cmems_model_v, cmems_data.depth, label='Model Datapoint', marker='x', color='black', s=100, alpha=1.0, zorder=3)
        axes[1,1].scatter(cmems_bin_v, cmems_bin_depths, label='1m Interpolation', color='orange', alpha=1.0, zorder=2)
        axes[1,1].axvline(x=cmems_avg_v, label=f'Depth Avgerage = [{cmems_avg_v:.2f}]', color='darkorange', linestyle='--', linewidth=2, zorder=1)
        axes[1,1].set_xlabel('v Velocity (m/s)', fontsize=12, fontweight='bold')
        axes[1,1].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[1,1].invert_yaxis()
        
        # MAGNITUDE PROFILE
        axes[1,2].scatter(cmems_bin_mag, cmems_bin_depths, label='1m Interpolation', color='green', alpha=1.0, zorder=2)
        axes[1,2].set_xlabel('Current Magnitude (m/s)', fontsize=12, fontweight='bold')
        axes[1,2].axvline(x=cmems_avg_mag, label=f'Depth Avgerage = [{cmems_avg_mag:.2f}]', color='darkgreen', linestyle='--', linewidth=2, zorder=1)
        axes[1,2].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[1,2].invert_yaxis()

        # DIRECTION PROFILE
        axes[1,3].scatter(cmems_bin_dir, cmems_bin_depths, label='1m Interpolation', color='purple', alpha=1.0, zorder=2)
        axes[1,3].set_xlabel('Current Direction (degrees)', fontsize=12, fontweight='bold')
        axes[1,3].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[1,3].invert_yaxis()

        # THRESHOLDS
        shading_colors = ['cyan', 'orange', 'lawngreen']
        for i, (data_1d, color) in enumerate(zip([cmems_bin_u1d, cmems_bin_v1d, cmems_bin_mag1d], shading_colors)):
            plot_profile_thresholds(axes[1, i], data_1d, threshold, color)

        # LEGEND
        for row in axes:
            for ax in row:
                handles, labels = ax.get_legend_handles_labels()
                if labels:
                    ax.legend(handles, labels, loc='lower center', facecolor='lightgrey', edgecolor='black', framealpha=1.0)

    if gofs_datasets is not None:
        # U-VELOCITY PROFILE
        axes[2,0].scatter(gofs_model_u, gofs_data.depth, marker='x', color='black', s=100, label='Model Datapoint', alpha=1.0, zorder=3)
        axes[2,0].scatter(gofs_bin_u, gofs_bin_depths, label='1m Interpolation', color='cyan', alpha=1.0, zorder=2)
        axes[2,0].axvline(x=gofs_avg_u, label=f'Depth Average = [{gofs_avg_u:.2f}]', color='darkcyan', linestyle='--', linewidth=2, zorder=1)
        axes[2,0].set_xlabel('u Velocity (m/s)', fontsize=12, fontweight='bold')
        axes[2,0].set_ylabel('Depth (m)', fontsize=12, fontweight='bold')
        axes[2,0].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[2,0].invert_yaxis()

        # V-VELOCITY PROFILE
        axes[2,1].scatter(gofs_model_v, gofs_data.depth, label='Model Datapoint', marker='x', color='black', s=100, alpha=1.0, zorder=3)
        axes[2,1].scatter(gofs_bin_v, gofs_bin_depths, label='1m Interpolation', color='orange', alpha=1.0, zorder=2)
        axes[2,1].axvline(x=gofs_avg_v, label=f'Depth Avgerage = [{gofs_avg_v:.2f}]', color='darkorange', linestyle='--', linewidth=2, zorder=1)
        axes[2,1].set_xlabel('v Velocity (m/s)', fontsize=12, fontweight='bold')
        axes[2,1].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[2,1].invert_yaxis()

        # MAGNITUDE PROFILE
        axes[2,2].scatter(gofs_bin_mag, gofs_bin_depths, label='1m Interpolation', color='green', alpha=1.0, zorder=2)
        axes[2,2].set_xlabel('Current Magnitude (m/s)', fontsize=12, fontweight='bold')
        axes[2,2].axvline(x=gofs_avg_mag, label=f'Depth Avgerage = [{gofs_avg_mag:.2f}]', color='darkgreen', linestyle='--', linewidth=2, zorder=1)
        axes[2,2].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[2,2].invert_yaxis()
        
        # DIRECTION PROFILE
        axes[2,3].scatter(gofs_bin_dir, gofs_bin_depths, label='1m Interpolation', color='purple', alpha=1.0, zorder=2)
        axes[2,3].set_xlabel('Current Direction (degrees)', fontsize=12, fontweight='bold')
        axes[2,3].grid(color='lightgrey', linestyle='-', linewidth=0.5)
        axes[2,3].invert_yaxis()

        # THRESHOLDS
        shading_colors = ['cyan', 'orange', 'lawngreen']
        for i, (data_1d, color) in enumerate(zip([gofs_bin_u1d, gofs_bin_v1d, gofs_bin_mag1d], shading_colors)):
            plot_profile_thresholds(axes[2, i], data_1d, threshold, color)

        # LEGEND
        for row in axes:
            for ax in row:
                handles, labels = ax.get_legend_handles_labels()
                if labels:
                    ax.legend(handles, labels, loc='lower center', facecolor='lightgrey', edgecolor='black', framealpha=1.0)
    
    # TITLES
    title_text = f"Vertical Profile Subplots - (Lat: {latitude_qc:.3f}, Lon: {longitude_qc:.3f})"
    format_subplot_titles(fig, config, model_data=depth_average_rtofs, title=title_text)

    subplot_titles = ['RTOFS', 'CMEMS', 'GOFS']
    format_subplot_headers(axes, fig, subplot_titles)

    # SAVE & CLOSE
    file_datetime = format_save_datetime(depth_average_rtofs)
    fig_filename = f"GGS_Profiles_{config['max_depth']}m_{file_datetime}.png"
    fig_path = os.path.join(directory, fig_filename)
    fig.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # LOGGING
    end_time = print_endtime()
    print_runtime(start_time, end_time)

latitude_qc='20.30'
longitude_qc='-86.50'
rtofs_datasets=(rtofs_data, depth_average_rtofs, bin_average_rtofs)
cmems_datasets=(cmems_data, depth_average_cmems, bin_average_cmems)
gofs_datasets=(gofs_data, depth_average_gofs, bin_average_gofs)
GGS_plot_profiles(config, sub_directory_plots, latitude_qc, longitude_qc, threshold=0.5, rtofs_datasets=rtofs_datasets, cmems_datasets=cmems_datasets, gofs_datasets=gofs_datasets)
